In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [3]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_7월.csv", encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [4]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [5]:
df

,url,본문,org_idx,cleaning
0,https://blog.naver.com/kuna611/222835896356,#25일급여 #삼성카드 #특판적금 #서울행복신협 #구즉신협 #카카오뱅크8%적금 #새...,0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...
1,https://blog.naver.com/zjavbxjqnd/222835899401,일요일 주말 휴가가 생겨서\n오빠랑 급 안동으로 여행 다녀왔어용\n(*ᴗ͈ˬᴗ͈)ꕤ...,1,일요일 주말 휴가가 생겨서 오빠랑 급 안동으로 여행 다녀왔어용 하루 여행이라 많은 ...
2,https://blog.naver.com/roseyh78/222835897663,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데\n다행이 내가 가고 ...,2,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데 다행이 내가 가고 싶...
3,https://blog.naver.com/y_girok_/222835898249,안녕-!\n재미없었고 별 다른 일이 없어서\n조용했던 저번주와는 달리\n시끌벅적 우...,3,안녕 재미없었고 별 다른 일이 없어서 조용했던 저번주와는 달리 시끌벅적 우당탕탕 재...
4,https://blog.naver.com/anuyheel/222835899414,수영구에서 뭐 재미있는 거 배울만한 게 있을까?\n새로운 것을 도전해보면서 문화 생...,4,수영구에서 뭐 재미있는 거 배울만한 게 있을까 새로운 것을 도전해보면서 문화 생활이...
...,...,...,...,...
18984,https://blog.naver.com/so9650/222795579526,"01\n까마귀 (The Raven, 1943) / 앙리 조르주 클루조\n02\n오르...",18984,01 까마귀 The Raven 1943 앙리 조르주 클루조 02 오르페브르의 부두 ...
18985,https://blog.naver.com/ottffss758/222794556276,안녕하세요 쪼~~~니 입니다.!\n만약 안동에 하루의 시간이 있다면\n어디를 가면 ...,18985,안녕하세요 쪼 니 입니다 만약 안동에 하루의 시간이 있다면 어디를 가면 좋을 까요 ...
18986,https://blog.naver.com/sil8004/222761395361,선비촌과 더불어 입장권 한 장으로 둘러볼 수 있는 소수서원은\n한국 최초의 서원이자...,18986,선비촌과 더불어 입장권 한 장으로 둘러볼 수 있는 소수서원은 한국 최초의 서원이자 ...
18987,https://blog.naver.com/cool2404/222795339338,"기획부터 문을 열기까지 햇수로 6년, 2000일을 공들여 선보인 트리니티클럽이 이제...",18987,기획부터 문을 열기까지 햇수로 6년 2000일을 공들여 선보인 트리니티클럽이 이제 ...


In [6]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

100%|██████████| 18989/18989 [38:00<00:00,  8.33it/s]  


,url,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/kuna611/222835896356,#25일급여 #삼성카드 #특판적금 #서울행복신협 #구즉신협 #카카오뱅크8%적금 #새...,0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,[(25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대...
1,https://blog.naver.com/zjavbxjqnd/222835899401,일요일 주말 휴가가 생겨서\n오빠랑 급 안동으로 여행 다녀왔어용\n(*ᴗ͈ˬᴗ͈)ꕤ...,1,일요일 주말 휴가가 생겨서 오빠랑 급 안동으로 여행 다녀왔어용 하루 여행이라 많은 ...,"[(일요일 주말 휴가가 생겨서 오빠랑 급 안동으로 여행 다녀왔어용, 0, 34, N..."
2,https://blog.naver.com/roseyh78/222835897663,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데\n다행이 내가 가고 ...,2,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데 다행이 내가 가고 싶...,[(경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데 다행이 내가 가고...
3,https://blog.naver.com/y_girok_/222835898249,안녕-!\n재미없었고 별 다른 일이 없어서\n조용했던 저번주와는 달리\n시끌벅적 우...,3,안녕 재미없었고 별 다른 일이 없어서 조용했던 저번주와는 달리 시끌벅적 우당탕탕 재...,[(안녕 재미없었고 별 다른 일이 없어서 조용했던 저번주와는 달리 시끌벅적 우당탕탕...
4,https://blog.naver.com/anuyheel/222835899414,수영구에서 뭐 재미있는 거 배울만한 게 있을까?\n새로운 것을 도전해보면서 문화 생...,4,수영구에서 뭐 재미있는 거 배울만한 게 있을까 새로운 것을 도전해보면서 문화 생활이...,[(수영구에서 뭐 재미있는 거 배울만한 게 있을까 새로운 것을 도전해보면서 문화 생...
...,...,...,...,...,...
18984,https://blog.naver.com/so9650/222795579526,"01\n까마귀 (The Raven, 1943) / 앙리 조르주 클루조\n02\n오르...",18984,01 까마귀 The Raven 1943 앙리 조르주 클루조 02 오르페브르의 부두 ...,[(01 까마귀 The Raven 1943 앙리 조르주 클루조 02 오르페브르의 부...
18985,https://blog.naver.com/ottffss758/222794556276,안녕하세요 쪼~~~니 입니다.!\n만약 안동에 하루의 시간이 있다면\n어디를 가면 ...,18985,안녕하세요 쪼 니 입니다 만약 안동에 하루의 시간이 있다면 어디를 가면 좋을 까요 ...,"[(안녕하세요 쪼 니 입니다, 0, 13, None, []), (만약 안동에 하루의..."
18986,https://blog.naver.com/sil8004/222761395361,선비촌과 더불어 입장권 한 장으로 둘러볼 수 있는 소수서원은\n한국 최초의 서원이자...,18986,선비촌과 더불어 입장권 한 장으로 둘러볼 수 있는 소수서원은 한국 최초의 서원이자 ...,[(선비촌과 더불어 입장권 한 장으로 둘러볼 수 있는 소수서원은 한국 최초의 서원이...
18987,https://blog.naver.com/cool2404/222795339338,"기획부터 문을 열기까지 햇수로 6년, 2000일을 공들여 선보인 트리니티클럽이 이제...",18987,기획부터 문을 열기까지 햇수로 6년 2000일을 공들여 선보인 트리니티클럽이 이제 ...,[(기획부터 문을 열기까지 햇수로 6년 2000일을 공들여 선보인 트리니티클럽이 이...


In [7]:
import pickle
# 데이터프레임을 피클 파일로 저장함으로써 데이터타입 보존
with open('7월_문장분리.pkl', 'wb') as file:
    pickle.dump(df, file)

In [2]:
import pickle
# 저장된 피클 파일을 불러오기 익일 재실행 경우, 이 코드만 돌리면 시간절약 가능.
with open('7월_문장분리.pkl', 'rb') as file:
    df = pickle.load(file)
df

,url,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/kuna611/222835896356,#25일급여 #삼성카드 #특판적금 #서울행복신협 #구즉신협 #카카오뱅크8%적금 #새...,0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,[(25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대...
1,https://blog.naver.com/zjavbxjqnd/222835899401,일요일 주말 휴가가 생겨서\n오빠랑 급 안동으로 여행 다녀왔어용\n(*ᴗ͈ˬᴗ͈)ꕤ...,1,일요일 주말 휴가가 생겨서 오빠랑 급 안동으로 여행 다녀왔어용 하루 여행이라 많은 ...,"[(일요일 주말 휴가가 생겨서 오빠랑 급 안동으로 여행 다녀왔어용, 0, 34, N..."
2,https://blog.naver.com/roseyh78/222835897663,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데\n다행이 내가 가고 ...,2,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데 다행이 내가 가고 싶...,[(경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데 다행이 내가 가고...
3,https://blog.naver.com/y_girok_/222835898249,안녕-!\n재미없었고 별 다른 일이 없어서\n조용했던 저번주와는 달리\n시끌벅적 우...,3,안녕 재미없었고 별 다른 일이 없어서 조용했던 저번주와는 달리 시끌벅적 우당탕탕 재...,[(안녕 재미없었고 별 다른 일이 없어서 조용했던 저번주와는 달리 시끌벅적 우당탕탕...
4,https://blog.naver.com/anuyheel/222835899414,수영구에서 뭐 재미있는 거 배울만한 게 있을까?\n새로운 것을 도전해보면서 문화 생...,4,수영구에서 뭐 재미있는 거 배울만한 게 있을까 새로운 것을 도전해보면서 문화 생활이...,[(수영구에서 뭐 재미있는 거 배울만한 게 있을까 새로운 것을 도전해보면서 문화 생...
...,...,...,...,...,...
18984,https://blog.naver.com/so9650/222795579526,"01\n까마귀 (The Raven, 1943) / 앙리 조르주 클루조\n02\n오르...",18984,01 까마귀 The Raven 1943 앙리 조르주 클루조 02 오르페브르의 부두 ...,[(01 까마귀 The Raven 1943 앙리 조르주 클루조 02 오르페브르의 부...
18985,https://blog.naver.com/ottffss758/222794556276,안녕하세요 쪼~~~니 입니다.!\n만약 안동에 하루의 시간이 있다면\n어디를 가면 ...,18985,안녕하세요 쪼 니 입니다 만약 안동에 하루의 시간이 있다면 어디를 가면 좋을 까요 ...,"[(안녕하세요 쪼 니 입니다, 0, 13, None, []), (만약 안동에 하루의..."
18986,https://blog.naver.com/sil8004/222761395361,선비촌과 더불어 입장권 한 장으로 둘러볼 수 있는 소수서원은\n한국 최초의 서원이자...,18986,선비촌과 더불어 입장권 한 장으로 둘러볼 수 있는 소수서원은 한국 최초의 서원이자 ...,[(선비촌과 더불어 입장권 한 장으로 둘러볼 수 있는 소수서원은 한국 최초의 서원이...
18987,https://blog.naver.com/cool2404/222795339338,"기획부터 문을 열기까지 햇수로 6년, 2000일을 공들여 선보인 트리니티클럽이 이제...",18987,기획부터 문을 열기까지 햇수로 6년 2000일을 공들여 선보인 트리니티클럽이 이제 ...,[(기획부터 문을 열기까지 햇수로 6년 2000일을 공들여 선보인 트리니티클럽이 이...


In [3]:
# 데이터프레임을 1000개 단위로 나누기
chunk_size = 50
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [10]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_7월_전처리.csv", index=False, encoding='UTF-8')

In [4]:
import pandas as pd
df2 = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_7월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,0
1,근데 급 다음주부터 발령나서 다른 곳으로 출근한다,0
2,2 3일 전에 알려주고 갑자기 거기로 출근이야 좌천인가 걱정됨,0
3,잘 적응하라고 뇌물을 준비해야 하나 ㅋㅋ 갑자기 문자메세지로 삼성카드 5월 링크 이...,0
4,소소하게 가계부 적립할게 생겨서 기분 좋은 날 2 화요일 오래전 엄마 집 옆에 있는...,0
...,...,...
882221,원주캠프는 1989년 한살림 선언을 한다 풀한포기 벌레 한마리도 사람과 똑같이 존엄...,18569
882222,원주 협동조합 지학순주교와 장일순 선생의 도움과 지도로 원주 지역 소상공인 직장인 ...,18569
882223,협동조합이 발전하게 된 계기가 있었다,18569
882224,1972년 8월 남한강 유역에 이틀 동안 450mm의 물폭탄이 쏟아져 원주교구 관할...,18569


In [8]:
dfs['df_1']['split_list'].index

RangeIndex(start=0, stop=10, step=1)

In [7]:
df2['org_idx'].tolist()[-1]//50+2

372

In [6]:
df2 = df2[df2['org_idx'] < 18550]

In [8]:
for a in range(df2['org_idx'].tolist()[-1]//50+2,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_7월_전처리.csv", index=False, encoding='UTF-8')
df2

100%|██████████| 39/39 [02:50<00:00,  4.36s/it]


,split_str,org_idx
0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,0
1,근데 급 다음주부터 발령나서 다른 곳으로 출근한다,0
2,2 3일 전에 알려주고 갑자기 거기로 출근이야 좌천인가 걱정됨,0
3,잘 적응하라고 뇌물을 준비해야 하나 ㅋㅋ 갑자기 문자메세지로 삼성카드 5월 링크 이...,0
4,소소하게 가계부 적립할게 생겨서 기분 좋은 날 2 화요일 오래전 엄마 집 옆에 있는...,0
...,...,...
901123,현재 지하 1층에서 흙과몸 성긴 연결 촘촘한 관계 6 9 전시실 를 테마로 강요배 ...,18988
901124,그 외에 Big Man 다시 일어서는 몸 박치호 애도 상실의 끝에서 김수자 게르하르...,18988
901125,광양시 문화관광 061 797 3 광양시 관광안내소 광양 문화관광 1경 백운산 4대...,18988
901126,자막보기 공지사항 제11회 최수종과 함께하 동북아 LNG Hub 광양시립소년소녀합창...,18988


In [9]:
df2.isnull().sum()

split_str    2
org_idx      0
dtype: int64

In [10]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_7월_전처리.csv", index=False, encoding='UTF-8')